https://assaeunji.github.io/machine%20learning/2020-11-29-implicitfeedback/

# test 3주 정합성 검증
- 

# 추천 모델
- ALS MF, LMF, MP (총 3개)
- 총 3개의 추천을 진행하며 MF와 LMF 의 경우 콜드스타트 유저(신규 유저)인 경우 MP로 추천 진행

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import scipy.sparse as sp
from implicit.lmf import LogisticMatrixFactorization as LMF

/home/user_3/medistream-recsys/Script


# 1.Dataload

In [15]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)

100%|██████████| 5141/5141 [00:00<00:00, 748348.61it/s]


In [16]:
df = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [17]:
df['date_paid'] = pd.to_datetime(df['date_paid'])

In [18]:
df['date_paid'].max()

Timestamp('2022-09-13 08:59:21.151000+0000', tz='UTC')

In [19]:
split_day_list =  []
test_day_list = []

# 1일 test 날짜 확인
last_date_state = '2022-09-13'
split_day = pd.to_datetime(last_date_state)-relativedelta(months=37)
test_day = pd.to_datetime(last_date_state)
'''
마지막 날짜에서 개월 수를 자름 -> split_day
months 만 바꾸면 21번 분량이 나옴
'''

for i in range(0,21):
    sp_day = str((split_day-timedelta(days=i)).to_pydatetime().date())
    tt_day  = str((test_day-timedelta(days=i)).to_pydatetime().date())
    split_day_list.append(sp_day)
    test_day_list.append(tt_day)

In [20]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]
    
    def _entropy_diversity(self,rec_list):
        import six
        import math
        
        topn = len(rec_list[0]['items'])
        users = [i.get('id',None) for i in rec_list]
        sz = float(len(users)) * topn
        freq = {}
        for rec in rec_list:
            for r in rec['items']:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)
        ent = self._entropy_diversity(rec_list)
        
        return ndcg_score, ent

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score, ent_score = self._eval(gt_list, rec_list)
            print(f"NDCG: {ndcg_score:.6}")
            print(f"Entropy Diversity: {ent_score:.6} ")
        except Exception as e:
            print(e)


In [21]:
def module(df:pd.DataFrame(), split_date, test_date)->pd.DataFrame():
    
    # paid orders만 가져오기
    df['date_paid'] = pd.to_datetime(df['date_paid'])
    df_only_paid = df[~df['date_paid'].isna()]
    # 3개월치 데이터만 가져오기
    df_date = df_only_paid[df_only_paid['date_paid'] >= split_date]
    # 취소 안된 것만 가져오기
    complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
    # 도서 카테고리만 가져오기
    only_book = complete_df[complete_df['name'] == '도서']

    # 유저가 중복으로 아이템 구매 삭제
    df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
    df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)
    
    train = df_book[df_book['date_paid'] < test_date]
    test = df_book[df_book['date_paid'].dt.date == pd.to_datetime(test_date)]
    
    
    # test 만 있는 item 제거
    only_test_items = set(test.product_ids.unique())-set(train.product_ids.unique())
    if_prepro_test = test[~test['product_ids'].isin(only_test_items)]
    
    
    PdIds = train.product_ids.unique()

    PdIdToIndex = {}
    indexToPdId = {}

    colIdx = 0

    for PdId in PdIds:
        PdIdToIndex[PdId] = colIdx
        indexToPdId[colIdx] = PdId
        colIdx += 1

    userIds = train.customer_id.unique()

    userIdToIndex = {}
    indexToUserId = {}

    rowIdx = 0

    for userId in userIds:
        userIdToIndex[userId] = rowIdx
        indexToUserId[rowIdx] = userId
        rowIdx += 1
    rows = []
    cols = []
    vals = []

    for row in train.itertuples():
        rows.append(userIdToIndex[row.customer_id])
        cols.append(PdIdToIndex[row.product_ids])
        vals.append(1)

    purchase_sparse = sp.csr_matrix((vals, (rows, cols)), shape=(rowIdx,colIdx))

    matrix = purchase_sparse.todense()

    medistream_prediction_df = train[['date_created','regular_price','sale_price','three_months','product_ids','name_x']]
    medistream_prediction_preprop_df = medistream_prediction_df.drop_duplicates(subset=['product_ids'], ignore_index=True)
    medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])    
    
    als_model = ALS(factors=20,regularization=0.01, iterations = 50, random_state=42)
    alsmodel = als_model.fit(purchase_sparse)
    lmf_model = LMF(factors=20, regularization=0.001, iterations = 20, random_state=42)
    lmfmodel = lmf_model.fit(purchase_sparse)
    
    most_popular = train.groupby(['product_ids','name_x']).count()['customer_id'].reset_index()
    most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index
    

        # test 예측값, 이미 구매 했을 경우 제외
    als_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(als_predict_list):
        als_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값
    lmf_predict_list = []
    for user_id in test['customer_id'].unique():
        try:
            result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
            lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
        except:
            train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
            lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(lmf_predict_list):
        lmf_predict_list[idx]['items'] = pred_list['items'][:100]

    

        # test 예측값, 이미 구매 했을 경우 제외
    predict_popular_list = []
    for user_id in test['customer_id'].unique():
        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
        predict_popular_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                ]})

        # 100 개만 예측하기
    for idx, pred_list in enumerate(predict_popular_list):
        predict_popular_list[idx]['items'] = pred_list['items'][:100]
        
    # real test 
    ground_trues = []
    for user_id in test['customer_id'].unique():
        ground_trues.append({'id': user_id,\
        'items':list(test[test['customer_id']==user_id].product_ids)
        })
    # ALS 
    evaluator = CustomEvaluator()
    als = evaluator._eval(ground_trues, als_predict_list)
    
    # LMF 
    evaluator = CustomEvaluator()
    lmf = evaluator._eval(ground_trues, lmf_predict_list)
    # MP
    evaluator = CustomEvaluator()
    mp = evaluator._eval(ground_trues, predict_popular_list)
    
    # 인기도순
    medistream_popular_list = medistream_prediction_preprop_df.sort_values(by='three_months', ascending=False).index
    # 최신순
    medistream_latest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=False).index
    # 오랜된 순
    medistream_oldest_list = medistream_prediction_preprop_df.sort_values(by='date_created', ascending=True).index
    # 높은 가격 순
    medistream_high_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=False).index
    # 낮은 가격 순
    medistream_low_price_list = medistream_prediction_preprop_df.sort_values(by='sale_price', ascending=True).index
    # 이름 순
    medistream_name_sort_list = medistream_prediction_preprop_df.sort_values(by='name_x',ascending=True).index

    def medistream_prediction_method(predict_num:int ,medi_predict_list:list)->list:
        medistream_predict_list = []
        for user_id in test['customer_id'].unique():
            medistream_predict_list.append({'id':user_id ,'items':[medistream_prediction_preprop_df.product_ids.loc[num] \
                                                                           for num in medi_predict_list]})

        # 100 개만 예측하기
        for idx, pred_list in enumerate(medistream_predict_list):
            medistream_predict_list[idx]['items'] = pred_list['items'][:predict_num]

        return medistream_predict_list
    
    medistream_predict_popular_list = medistream_prediction_method(100, medistream_popular_list)
    medistream_predict_latest_list = medistream_prediction_method(100, medistream_latest_list)
    medistream_predict_oldest_list = medistream_prediction_method(100, medistream_oldest_list)
    medistream_predict_high_price_list = medistream_prediction_method(100, medistream_high_price_list)
    medistream_predict_low_price_list = medistream_prediction_method(100, medistream_low_price_list)
    medistream_predict_name_sort_list = medistream_prediction_method(100, medistream_name_sort_list)
    
    def medistream_prediction(ground_trues:list, predict_list:list):
        evaluator = CustomEvaluator()
        ndcg, entropy = evaluator._eval(ground_trues, predict_list)

        assert len(predict_list) == len(ground_trues)

        cnt = 0
        for gt, pred_list in zip(ground_trues, predict_list):
            for pred in pred_list['items']:
                if pred in gt['items']:
                    cnt += 1
        return ndcg, entropy, cnt
    
    medistream_predict_score = {'medistream_predict':['medi_popular','latest','oldest','high_price','low_price','name_sort'], \
                                'ndcg':[], 'entropy':[], 'cnt':[]}

    medistream_predict_list = [medistream_predict_popular_list, medistream_predict_latest_list, medistream_predict_oldest_list,\
                              medistream_predict_high_price_list, medistream_predict_low_price_list, medistream_predict_name_sort_list]

    for medistream_predict in medistream_predict_list:
        ndcg, entropy, cnt = medistream_prediction(ground_trues, medistream_predict)
        medistream_predict_score['ndcg'].append(ndcg)
        medistream_predict_score['entropy'].append(entropy)
        medistream_predict_score['cnt'].append(cnt)

        
######## hyper parameter
    als_mf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                als_model = ALS(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                als_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저인 경우 mp로 넣기
                # 전체 도서에 대한 판매 만큼 정렬 후 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값, 이미 구매 했을 경우 제외
                als_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = als_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        als_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        als_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(als_predict_list):
                    als_predict_list[idx]['items'] = pred_list['items'][:100]

                # ALS 
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, als_predict_list)

                als_mf_hyper_parameter['factor'].append(factor)
                als_mf_hyper_parameter['regularization'].append(regularization)
                als_mf_hyper_parameter['iteration'].append(iteration)
                als_mf_hyper_parameter['NDCG'].append(ndcg)
                als_mf_hyper_parameter['entropy'].append(entropy)
                
                
    lmf_hyper_parameter = {'factor':[],'regularization':[],'iteration':[],'NDCG':[],'entropy':[]}

    factors = [5,10,15,20]
    regularizations = [0.01,0.005]
    iterations = [5,10,15,20,25,30,40,50]

    for factor in factors:
        for regularization in regularizations:
            for iteration in iterations:
                lmf_model = LMF(factors=factor, regularization=regularization, iterations = iteration, random_state=42)
                lmf_model.fit(purchase_sparse, show_progress=False)

                # 신규 유저 mp로 넣기
                most_popular_list = most_popular.sort_values(by='customer_id',ascending=False).index

                # test 예측값
                lmf_predict_list = []
                for user_id in test['customer_id'].unique():
                    try:
                        result = lmf_model.recommend(userIdToIndex[user_id], purchase_sparse[userIdToIndex[user_id]], N=100)
                        lmf_predict_list.append({'id':user_id ,'items':[indexToPdId[num] for num in result[0]]})
                    except:
                        train_purchase_list = list(train[train['customer_id']==user_id].product_ids)
                        lmf_predict_list.append({'id':user_id ,'items':[most_popular.product_ids.loc[num] for num in most_popular_list \
                                                                            if most_popular.product_ids.loc[num] not in train_purchase_list \
                                                                            ]})

                # 100 개만 예측하기
                for idx, pred_list in enumerate(lmf_predict_list):
                    lmf_predict_list[idx]['items'] = pred_list['items'][:100]

                # LMF
                evaluator = CustomEvaluator()
                ndcg, entropy = evaluator._eval(ground_trues, lmf_predict_list)

                lmf_hyper_parameter['factor'].append(factor)
                lmf_hyper_parameter['regularization'].append(regularization)
                lmf_hyper_parameter['iteration'].append(iteration)
                lmf_hyper_parameter['NDCG'].append(ndcg)
                lmf_hyper_parameter['entropy'].append(entropy)
                
#     return als_mf_hyper_parameter, lmf_hyper_parameter, medistream_predict_score

    all_prediction_df = {'first_day':[],'last_day':[],'train_데이터수':[],'train_유저수':[],'test_데이터수':[],\
        'test_유저수':[],'test_신규유저수':[],'test_신규아이템수':[],'원본_test수':[],'전처리진행test수':[],\
        'als_mf':[],'lmf':[],'mp':[],'medi_popular':[],'latest':[],\
        'oldest':[],'high_price':[],'low_price':[],'name_sort':[],\
         'als_mf_entropy':[],'lmf_entropy':[],'mp_entropy':[],'medi_popular_entropy':[],'latest_entropy':[],\
         'oldest_entropy':[],'high_price_entropy':[],'low_price_entropy':[],'name_sort_entropy':[]}
    medistream_predict_df = pd.DataFrame(medistream_predict_score)

    all_prediction_df['first_day'].append(str(datetime.date(train['date_paid'].min()))+' '+str(datetime.date(train['date_paid'].max())))
    all_prediction_df['last_day'].append(str(datetime.date(test['date_paid'].min()))+' '+str(datetime.date(test['date_paid'].max())))
    all_prediction_df['train_데이터수'].append(len(train))
    all_prediction_df['train_유저수'].append(len(set(train.customer_id)))
    all_prediction_df['test_데이터수'].append(len(test))
    all_prediction_df['test_유저수'].append(len(set(test.customer_id)))
    all_prediction_df['test_신규유저수'].append(len(set(test['customer_id'].unique())- set(train['customer_id'].unique())))
    all_prediction_df['test_신규아이템수'].append(len(set(test.product_ids.unique())-set(train.product_ids.unique())))
    all_prediction_df['원본_test수'].append(len(test))
    all_prediction_df['전처리진행test수'].append(len(if_prepro_test))

    # ndcg
    all_prediction_df['als_mf'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['lmf'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='NDCG',ascending=False)['NDCG'].iloc[0])
    all_prediction_df['mp'].append(evaluator._eval(ground_trues, predict_popular_list)[0])
    all_prediction_df['medi_popular'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['ndcg'])
    all_prediction_df['latest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['ndcg'])
    all_prediction_df['oldest'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['ndcg'])
    all_prediction_df['high_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['ndcg'])
    all_prediction_df['low_price'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['ndcg'])
    all_prediction_df['name_sort'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['ndcg'])

#     entropy
    all_prediction_df['als_mf_entropy'].append(pd.DataFrame(als_mf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['lmf_entropy'].append(pd.DataFrame(lmf_hyper_parameter).sort_values(by='entropy',ascending=False)['entropy'].iloc[0])
    all_prediction_df['mp_entropy'].append(evaluator._eval(ground_trues, predict_popular_list)[1])
    all_prediction_df['medi_popular_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='medi_popular'].iloc[0]['entropy'])
    all_prediction_df['latest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='latest'].iloc[0]['entropy'])
    all_prediction_df['oldest_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='oldest'].iloc[0]['entropy'])
    all_prediction_df['high_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='high_price'].iloc[0]['entropy'])
    all_prediction_df['low_price_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='low_price'].iloc[0]['entropy'])
    all_prediction_df['name_sort_entropy'].append(medistream_predict_df[medistream_predict_df['medistream_predict']=='name_sort'].iloc[0]['entropy'])

    print('train 총 기간:',train['date_paid'].max()-train['date_paid'].min())
    print('test 총 기간:',test['date_paid'].max()-test['date_paid'].min())
    
    return pd.DataFrame(all_prediction_df)

In [22]:
from tqdm import tqdm

twenty_df_list = []
for sp_day, tt_day in tqdm(zip(split_day_list,test_day_list)):
    module_df = module(df, sp_day,tt_day)
    twenty_df_list.append(module_df)
twenty_df = pd.concat(twenty_df_list, ignore_index=True)
twenty_df

0it [00:00, ?it/s]<ipython-input-21-b3ae7dbb3584>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medistream_prediction_preprop_df['date_created'] = pd.to_datetime(medistream_prediction_preprop_df['date_created'])


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

1it [04:54, 294.88s/it]

train 총 기간: 1085 days 23:08:32.448000
test 총 기간: 0 days 08:44:11.628000


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

21it [2:15:20, 386.69s/it]

train 총 기간: 1065 days 23:26:33.394000
test 총 기간: 0 days 13:49:59


,first_day,last_day,train_데이터수,train_유저수,test_데이터수,test_유저수,test_신규유저수,test_신규아이템수,원본_test수,전처리진행test수,als_mf,lmf,mp,medi_popular,latest,oldest,high_price,low_price,name_sort,als_mf_entropy,lmf_entropy,mp_entropy,medi_popular_entropy,latest_entropy,oldest_entropy,high_price_entropy,low_price_entropy,name_sort_entropy
0,2019-09-23 2022-09-12,2022-09-13 2022-09-13,37850,7409,16,13,1,1,16,15,0.257257,0.286001,0.348660,0.212806,0.110887,0.100933,0.037917,0.060388,0.099823,5.389125,5.522476,4.728534,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
1,2019-09-23 2022-09-11,2022-09-12 2022-09-12,37835,7406,15,11,3,0,15,15,0.297711,0.245029,0.227500,0.292188,0.065425,0.133306,0.082232,0.019405,0.035447,5.183278,5.411535,4.776034,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
2,2019-09-23 2022-09-10,2022-09-11 2022-09-11,37831,7406,4,2,0,0,4,4,0.154827,0.240090,0.425172,0.425172,0.082848,0.000000,0.193426,0.072170,0.058004,5.090373,5.062647,4.612102,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
3,2019-09-23 2022-09-09,2022-09-10 2022-09-10,37829,7406,2,1,0,0,2,2,0.403287,1.000000,0.226957,0.229675,0.000000,0.200249,0.095413,0.102977,0.000000,4.605170,4.605170,4.577444,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
4,2019-09-23 2022-09-08,2022-09-09 2022-09-09,37823,7406,6,4,0,0,6,6,0.307050,0.264646,0.191046,0.162690,0.025453,0.210808,0.091671,0.023072,0.000000,5.187007,5.359580,4.680191,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
5,2019-09-23 2022-09-07,2022-09-08 2022-09-08,37724,7395,99,81,11,0,99,99,0.187531,0.193552,0.232427,0.398146,0.396668,0.036035,0.015691,0.020938,0.020789,5.392370,5.530995,4.724608,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
6,2019-09-23 2022-09-06,2022-09-07 2022-09-07,37691,7393,33,22,2,0,33,33,0.244371,0.274589,0.223568,0.299082,0.142594,0.102501,0.014464,0.070644,0.033362,5.387111,5.501660,4.718836,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
7,2019-09-23 2022-09-05,2022-09-06 2022-09-06,37633,7390,58,50,3,0,58,58,0.113358,0.104374,0.076439,0.291598,0.079309,0.126021,0.001778,0.009945,0.000000,5.393536,5.562302,4.818455,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
8,2019-09-23 2022-09-04,2022-09-05 2022-09-05,37609,7384,24,18,6,0,24,24,0.275464,0.318540,0.311774,0.329348,0.195413,0.069962,0.030477,0.045998,0.042236,5.256010,5.415130,4.712613,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
9,2019-09-23 2022-09-03,2022-09-04 2022-09-04,37589,7383,20,15,1,0,20,20,0.362326,0.229839,0.329322,0.267457,0.120659,0.163072,0.115002,0.011046,0.085243,5.386794,5.455410,4.719519,4.60517,4.60517,4.60517,4.60517,4.60517,4.60517
